<img src="https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Vectorización


In [10]:
import numpy as np
import pandas as pd

In [2]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * (np.linalg.norm(b)))

### Datos

In [3]:
corpus = np.array(['que dia es hoy', 'martes el dia de hoy es martes', 'martes muchas gracias'])

Documento 1 --> que dia es hoy \
Documento 2 --> martes el dia de hoy es martes \
Documento 3 --> martes muchas gracias

### 1 - Obtener el vocabulario del corpus (los términos utilizados)
- Cada documento transformarlo en una lista de términos
- Armar un vector de términos no repetidos de todos los documentos

In [4]:
# Se genera una lista de listas de terminos
terminos = []
for c in corpus:
  terminos.append(c.split())

terminos

[['que', 'dia', 'es', 'hoy'],
 ['martes', 'el', 'dia', 'de', 'hoy', 'es', 'martes'],
 ['martes', 'muchas', 'gracias']]

In [5]:
# Se hace un flatten de las listas anidadas
terminos_set = [item for sublist in terminos for item in sublist]

# Se castea a set para eliminar duplicados
terminos_set = set(terminos_set)
terminos_set

{'de', 'dia', 'el', 'es', 'gracias', 'hoy', 'martes', 'muchas', 'que'}

In [6]:
def get_terminos_set(corpus:np.array)->set:
  '''
  Metodo para extraer lista de terminos unicos a partir de un corpus

  Parameters:
  -----------
    - corpus (np.array): array de documentos

  Return:
  -------
    - terminos_set (set): set de terminos de todos los documentos
  '''

  terminos = []
  for c in corpus:
    terminos.append(c.split())

  terminos_set = [item for sublist in terminos for item in sublist]
  terminos_set = set(terminos_set)
  return terminos_set

### 2- OneHot encoding
Data una lista de textos, devolver una matriz con la representación oneHotEncoding de estos

In [7]:
def get_oneHotEnc(corpus:np.array)->np.array:
  '''
  Metodo para calcular matriz de one hot encoding de los documentos de un corpus

  Parameters:
  -----------
    - corpus (np.array): array de documentos

  Return:
  -------
    - one_hots (np.array): array de vectores one hot, uno para cada documento
  '''

  terminos_set = get_terminos_set(corpus)

  one_hots = []
  for c in corpus:
    one_hot = []
    for t in terminos_set:
      if ' '+t+' ' in ' '+c+' ':
        one_hot.append(1)
      else:
        one_hot.append(0)
    one_hots.append(np.array(one_hot))

  return np.array(one_hots)

In [8]:
# Calculo de vectores one-hot
m = get_oneHotEnc(corpus)
m

array([[1, 0, 0, 1, 1, 0, 1, 0, 0],
       [1, 0, 1, 0, 1, 0, 1, 1, 1],
       [0, 1, 0, 0, 0, 1, 0, 0, 1]])

In [11]:
# Se genera un pd.DataFrame para visualizar mejor
df = pd.DataFrame(m,columns=list(terminos_set),index=corpus)
df

,hoy,gracias,el,que,dia,muchas,es,de,martes
que dia es hoy,1,0,0,1,1,0,1,0,0
martes el dia de hoy es martes,1,0,1,0,1,0,1,1,1
martes muchas gracias,0,1,0,0,0,1,0,0,1


### 3- Vectores de frecuencia
Data una lista de textos, devolver una matriz con la representación de frecuencia de estos

In [19]:
def get_freq_vects(corpus:np.array)->[np.array,set]:
  '''
  Metodo para calcular vectores de frecuencia de los documentos de un corpus

  Parameters:
  -----------
    - corpus (np.array): array de documentos

  Return:
  -------
    - freqs (np.array): array de vectores de frecuencia, uno para cada documento
    - terminos_set (set): set de terminos unicos de todos los documentos
  '''

  # Obtencion de terminos unicos del corpus
  terminos_set = get_terminos_set(corpus)

  # Se recorre cada documento del corpus
  freqs = []
  for c in corpus:
    freq = []
    # Se contabilizan ocurrencias de terminos unicos
    for t in terminos_set:
      freq.append(c.split().count(t))

    freqs.append(np.array(freq))

  return np.array(freqs), terminos_set

In [20]:
# Calculo de vectores de frecuencia
freqs, _ = get_freq_vects(corpus)
freqs

array([[1, 0, 0, 1, 1, 0, 1, 0, 0],
       [1, 0, 1, 0, 1, 0, 1, 1, 2],
       [0, 1, 0, 0, 0, 1, 0, 0, 1]])

In [21]:
# Se genera un pd.DataFrame para visualizar mejor
df = pd.DataFrame(freqs,columns=list(terminos_set),index=corpus)
df

,hoy,gracias,el,que,dia,muchas,es,de,martes
que dia es hoy,1,0,0,1,1,0,1,0,0
martes el dia de hoy es martes,1,0,1,0,1,0,1,1,2
martes muchas gracias,0,1,0,0,0,1,0,0,1


### 4- TF-IDF
Data una lista de textos, devolver una matriz con la representacion TFIDF

In [22]:
def get_idf(corpus:np.array, term:str)->float:
  '''
  Metodo para calcular el idf de un termino respecto de un corpus

  Parameters:
  -----------
    - corpus (np.array): array de documentos
    - term (str): termino al que se le quiere calcular el idf

  Return:
  -------
    - idf (float): inverse document frequency
  '''

  # Numero total de documentos del corpus
  N = corpus.shape[0]
  # Inicializamos en 0 el contador de documentos
  DF = 0

  # Se recorre cada documento del corpus
  for c in corpus:
    # Y se suma uno si el termino en cuestion figura en el documento
    if ' '+term+' ' in ' '+c+' ':
      DF +=1

  # Se calcula el idf como el log10 de la razon N/DF
  idf = np.log10(N/DF)
  return idf


In [24]:
# Corrida de ejemplo
get_idf(corpus,'gracias')

0.47712125471966244

In [25]:
def get_tf_idf(corpus:np.array)->[np.array,set]:
  '''
  Metodo para calcular vectores tf-idf para cada documento

  Parameters:
  -----------
    - corpus (np.array): array de documentos

  Return:
  -------
    - tf_idf (np.array): array de vectores tf-idf, uno por documento
    - terminos_set (set): set de terminos unicos del corpus
  '''

  freqs, terminos_set = get_freq_vects(corpus)

  idfs = [get_idf(corpus,t) for t in terminos_set]

  tf_idf = freqs * idfs
  return tf_idf, terminos_set

In [26]:
tf_idf, terminos_set = get_tf_idf(corpus)
tf_idf

array([[0.17609126, 0.        , 0.        , 0.47712125, 0.17609126,
        0.        , 0.17609126, 0.        , 0.        ],
       [0.17609126, 0.        , 0.47712125, 0.        , 0.17609126,
        0.        , 0.17609126, 0.47712125, 0.35218252],
       [0.        , 0.47712125, 0.        , 0.        , 0.        ,
        0.47712125, 0.        , 0.        , 0.17609126]])

In [27]:
# Se genera un pd.DataFrame para visualizar mejor
df = pd.DataFrame(tf_idf,columns=list(terminos_set),index=corpus)
df

,hoy,gracias,el,que,dia,muchas,es,de,martes
que dia es hoy,0.176091,0.000000,0.000000,0.477121,0.176091,0.000000,0.176091,0.000000,0.000000
martes el dia de hoy es martes,0.176091,0.000000,0.477121,0.000000,0.176091,0.000000,0.176091,0.477121,0.352183
martes muchas gracias,0.000000,0.477121,0.000000,0.000000,0.000000,0.477121,0.000000,0.000000,0.176091


### 5 - Comparación de documentos
Realizar una funcion que reciba el corpus y el índice de un documento y devuelva los documentos ordenados por la similitud coseno

In [28]:
def get_simil(corpus:np.array, idx:int)->list:
  '''
  Metodo para calcular similaridad de documentos de un corpus

  Parameters:
  -----------
    - corpus (np.array): array de documentos
    - idx (int): indice del documento del corpus que se quiere analizar

  Return:
  -------
    - simils (list): lista de tuplas con score de similaridad y documentos
  '''

  # Se obtienen los vectores de tf-idf para cada documento, y set de terminos
  tf_idf, terminos_set = get_tf_idf(corpus)
  # Se crea un pd.DataFrame para indexar los vectores mejor
  df = pd.DataFrame(tf_idf,columns=list(terminos_set),index=corpus)

  # Se separa el documento a analizar del resto del corpus y se toma su vector
  doc = corpus[idx]
  doc_vec = df.loc[doc].values
  corpus = corpus[corpus!=doc]

  # Como primer elemento se coloca el documento a analizar con similaridad 1
  simils = [[1,doc]]
  # Se recorre el resto del corpus
  for c in corpus:
    # Se toma cada vector tf-idf
    c_vec = df.loc[c].values
    # Se calcula la similaridad coseno
    simil = cosine_similarity(doc_vec,c_vec)
    # Se agrega a la lista respuesta el score de similaridad y el documento
    simils.append([simil,c])

  # Se ordena la lista por similaridad, de mayor a menor
  simils.sort(key=lambda x: x[0], reverse=True)
  return simils

In [29]:
# Se prueba el metodo
get_simil(corpus, 2)

[[1, 'martes muchas gracias'],
 [0.10845711727883083, 'martes el dia de hoy es martes'],
 [0.0, 'que dia es hoy']]

In [33]:
# Se genera un corpus para validar el método

corpus1 = np.array(['La tecnologia es una herramienta en si misma',
                   'Con una herramienta uno puede construir cosas',
                   'si sale el sol va a estar lindo',
                   'un día lindo es un buen día',
                   'el perro es feliz en un sillón un día soleado'])

In [34]:
get_simil(corpus1,0)

[[1, 'La tecnologia es una herramienta en si misma'],
 [0.13007684606237055, 'Con una herramienta uno puede construir cosas'],
 [0.08027967939282979, 'el perro es feliz en un sillón un día soleado'],
 [0.06324894545860953, 'si sale el sol va a estar lindo'],
 [0.023966598430926654, 'un día lindo es un buen día']]

In [35]:
get_simil(corpus1,3)

[[1, 'un día lindo es un buen día'],
 [0.40433864229842487, 'el perro es feliz en un sillón un día soleado'],
 [0.06616746575212422, 'si sale el sol va a estar lindo'],
 [0.023966598430926654, 'La tecnologia es una herramienta en si misma'],
 [0.0, 'Con una herramienta uno puede construir cosas']]

**Conclusiones:**

Se evalúa el desempeño del mecanismo de TF-IDF para establecer la similaridad de documentos en un corpus.

Como principal ventaja se puede destacar la velocidad y la posibilidad de implementarlo con pocas librerías.

Y como principal desventaja está el hecho de que se basa en la ocurrencia de palabras en el documento. Por un lado, no concidera errores en las palabras y por otro no se captura ningún "fenómeno" gramatical, como sustantivos, adjetivos, intenión, etc.